---
Some useful $\LaTeX$ commands are defined in this cell:
$$
\newcommand{\abs}[1]{\left\lvert#1\right\rvert}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\set}[1]{\left\{#1\right\}}
\newcommand{\paren}[1]{\left(#1\right)}
\newcommand{\brack}[1]{\left[#1\right]}
\newcommand{\ip}[2]{\left\langle#1,#2\right\rangle}
\DeclareMathOperator{\span}{span}
\DeclareMathOperator{\fl}{fl}
\abs{x}, \norm{x}, \set{x}, \paren{x}, \brack{x}, \ip{x}{y}, \span, \fl
$$

---

---
# 15.4 Adaptive quadrature
---

## Example

Let 

$$f(x) = e^{-3x} \sin 4x$$

and suppose we need to calculate $\int_0^4 f(x)\,dx$.

In [ ]:
f(x) = exp(-3x).*sin(4x)

In [ ]:
using Plots, LaTeXStrings

a, b = 0, 4

xx = range(a, b, length=200)

plot(legend=:none)
plot!(xx, f.(xx), ribbon=(f.(xx), zeros(length(xx))))
hline!([0], c=:black)

Notice how $f(x)$ varies dramatically for $x < 1$ and then has very small variation for $x > 1$.

It would be better to divide the interval $[0,4]$ into many smaller subintervals in the region $[0,1]$ and fewer subintervals in the region $[1,4]$.

We will do this subdivision **adaptively**, only when it helps improve the accuracy of numerical integration.

To do this, we need to obtain a good estimate of the error.

---

## Computing an error estimate via Richardson extrapolation

Let $I_f = \int_a^b f(x) \,dx$ and consider the **Simpson rule** 

$$S(a,b) = \frac{h}{3} \brack{f(a) + 4f(a+h) + f(b)},$$

where $h = (b - a)/2$. 

Let $S_1 = S(a,b)$.

Since the composite Simpson rule is a **fourth-order accurate** method, we have

$$I_f = S_1 + Kh^4 + \mathcal{O}(h^5).$$

Now consider using the **composite Simpson rule** with a step-size of $h/2$ on the subintervals $[a,a+h]$ and $[a+h,b]$:

$$
\begin{align}
S(a,a+h) &= \frac{h}{6}\brack{f(a) + 4f(a + h/2) + f(a + h)},\\
\\
S(a+h,b) &= \frac{h}{6}\brack{f(a+h) + 4f(a + 3h/2) + f(b)}.\\
\end{align}
$$

Let $S_2 = S(a,a+h) + S(a+h,b)$.

Then we have

$$I_f = S_2 + K\paren{\frac{h}{2}}^4 + \mathcal{O}(h^5).$$

Therefore,

$$S_1 + Kh^4 + \mathcal{O}(h^5) = S_2 + K\paren{\frac{h}{2}}^4 + \mathcal{O}(h^5).$$

Then we solve for the error term $Kh^4$:

$$Kh^4 = \frac{16}{15}\brack{S_2 - S_1} + \mathcal{O}(h^5).$$

From this we conclude that

$$ 
\begin{align}
I_f - S_1 &= \frac{16}{15}\brack{S_2 - S_1} + \mathcal{O}(h^5),\\
\\
I_f - S_2 &= \frac{1}{15}\brack{S_2 - S_1} + \mathcal{O}(h^5).\\
\end{align}
$$

---

## Example

Consider

$$I_f = \int_{-1}^1 \cos\paren{\frac{\pi}{2} x} \,dx = \frac{4}{\pi} = \underline{1.2732395447351628} \ldots .$$

The **Simpson rule** gives us:

$$S_1 = \frac{2}{6}\paren{0 + 4 \cdot 1 + 0} = \frac{4}{3} =  \underline{1.3}333333333333333 \ldots .$$

Thus, the error is 

$$\abs{I_f - S_1} = 0.060093788598170494\ldots.$$

In [ ]:
f(x) = cos(pi/2*x)

In [ ]:
If = 4/pi
S1 = (f(-1) + 4f(0) + f(1))/3

@show If
@show S1
@show abs(If - S1);

Now let's compute $S_2$:

$$
\begin{align}
S_2 
&= \frac{1}{6}\paren{f(-1) + 4f(-.5) + 2f(0) + 4f(.5) + f(1)} \\
&= \frac{1}{6}\paren{0 + \frac{4}{\sqrt{2}} + 2 + \frac{4}{\sqrt{2}} + 0} \\
&= \underline{1.27}61423749153966\ldots.\\
\end{align}
$$

The error is

$$\abs{I_f - S_2} = 0.0029028301802338508\ldots.$$

In [ ]:
S2 = (f(-1) + 4f(-.5) + 2f(0) + 4f(.5) + f(1))/6

@show If
@show S2
@show abs(If - S2);

Now, let's compare our error estimates. We have

$$
\begin{align}
\abs{I_f - S_1} &\approx \frac{16}{15}\abs{S_2 - S_1} = \underline{0.06}100368897913242\ldots,\\
\\
\abs{I_f - S_2} &\approx \frac{1}{15}\abs{S_2 - S_1} = \underline{0.003}8127305611957763\ldots.\\
\end{align}
$$

Thus, the error estimates are fairly accurate.

In [ ]:
@show 16/15*abs(S2 - S1)
@show 1/15*abs(S2 - S1);

---

## Divide-and-conquer

We now describe a **divide-and-conquer** approach to obtain a quadrature approximation $Q_f$ of $I_f = \int_a^b f(x)\,dx$ such that

$$\abs{Q_f - I_f} < \mathtt{tol},$$

where $\mathtt{tol}$ is some user-specified tolerance.

The idea is to do an **adaptive local refinement** of the grid of points on which we perform the composite Simpson rule (or any other quadrature rule):

$$a = t_0 < t_1 < \cdots < t_r = b.$$

Over each subinterval $[t_{i-1}, t_i]$, we compute $Q_i \approx I_i = \int_{t_{i-1}}^{t_i} f(x)\,dx$ such that

$$\abs{Q_i - I_i} < \frac{h_i}{b-a}\mathtt{tol},$$

where $h_i = t_i - t_{i-1}$, and then let

$$Q_f = \sum_{i=1}^r Q_i.$$

Then,

$$\begin{split}
\abs{Q_f - I_f}
&= \abs{\sum_{i=1}^r Q_i - \sum_{i=1}^r I_i} \\
&= \abs{\sum_{i=1}^r (Q_i - I_i)} \\
&\leq \sum_{i=1}^r \abs{ Q_i - I_i} \qquad \text{(by the Triangle Inequality)}\\
&< \sum_{i=1}^r \frac{h_i}{b - a} \mathtt{tol} \\
&= \frac{\mathtt{tol}}{b - a}  \sum_{i=1}^r h_i \\
&= \mathtt{tol}.
\end{split}
$$

Thus, we just need to check that our error estimate for the current subinterval is small enough.

If the error estimate is not small enough, we divide the current subinterval into two equal pieces and repeat (recursively).

---

## A simple recursive implementation

In [ ]:
############################################################
function Simp1(f::Function, a::Float64, b::Float64)
    h = (b - a)/2
    return h/3*(f(a) + 4f(a + h) + f(b))
end

############################################################
Simp2(f, a::Float64, b::Float64) = Simp1(f, a, (a+b)/2) + Simp1(f, (a+b)/2, b)

############################################################
function quadsimp(f::Function, a::Float64, b::Float64, tol::Float64)
    
    h = (b - a)/2
    
    S1 = Simp1(f, a, b)
    S2 = Simp2(f, a, b)
    
    E2 = abs(S2 - S1)/15
    
    if E2 < tol
        Q = S2
        mesh = [a, a+h/2, a+h, a+3h/2, b]
    else
        Q1, mesh1 = quadsimp(f, a, (a+b)/2, tol/2)
        Q2, mesh2 = quadsimp(f, (a+b)/2, b, tol/2)
        Q = Q1 + Q2
        mesh = vcat(mesh1, mesh2[2:end])
    end
    
    return Q, mesh
end

In [ ]:
f(x) = exp(-3x).*sin(4x)

In [ ]:
using SymPy

intval = Float64(integrate(f, a, b))

In [ ]:
integrate(f, a, b)

In [ ]:
a, b, tol = 0., 4., 1e-4
Q, mesh = quadsimp(f, a, b, tol)

@show tol
@show abs(intval - Q);

In [ ]:
xx = range(a, b, length=200)

plot(legend=:none, title="Adaptive quadrature")
plot!(xx, f.(xx), ribbon=(f.(xx), zeros(length(xx))))
hline!([0], c=:black)
plot!(mesh, zeros(length(mesh)), m=:+, c=:red)

In [ ]:
using Printf

tols = [10.0^(-k) for k=1:16]

quads = Float64[]
fevals = Int64[]
tt = Float64[]
for tol in tols
    t = @elapsed Q, mesh = quadsimp(f, a, b, tol)
    push!(quads, Q)
    push!(fevals, length(mesh))
    push!(tt, t)
end
    
abserr = abs.(quads .- intval)

@printf "%6s %10s %7s %10s\n" "tol" "abserr" "fevals" "time"
for k = 1:length(tols)
    @printf "%6.0e %10.2e %7d %10.6f\n" tols[k] abserr[k] fevals[k] tt[k]
end

---

## Example

The number of prime numbers less or equal to $x$ is denoted $\pi(x)$.

The [prime number theorem](https://en.wikipedia.org/wiki/Prime_number_theorem#Prime-counting_function_in_terms_of_the_logarithmic_integral) states that

$$
\pi(x) \sim \int_2^x \frac{dt}{\log t}.
$$

In [ ]:
using Primes

x = 200

primes(x)

In [ ]:
f(t) = 1/log(t)
a, b, tol = 2., Float64(x), 1e-6

Q, mesh = quadsimp(f, a, b, tol)

length(primes(x)), Q, length(mesh)

In [ ]:
n = 10000

px = [length(primes(k)) for k=2:n];
Li = [quadsimp(f, 2., Float64(k), tol)[1] for k=2:n];

plot(legend=:topleft)
#plot!(px, label=L"\pi(x)")
#plot!(Li, label=L"\mathrm{Li}(x)")
plot!(Li./px, label=L"\mathrm{Li}(x)/\pi(x)")

---

In [ ]:
using QuadGK

In [ ]:
?quadgk

---